In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
filepath ='drive/MyDrive/hofstra/HitData/Hit215/ADV DA Project/Adv data project/Datasets/'
list_of_files = os.listdir(filepath)
print(list_of_files)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['ADMISSIONS.csv', 'PRESCRIPTION.csv', 'heart_failure_clinical_records_dataset.csv', 'Heart_health.csv', 'cancer_classification.csv', 'diabetes_risk_prediction_dataset.csv', 'fetal_health.csv', 'pres.csv', 'Patient Admissions dummi data.csv', 'LAB.csv', 'LABEV.csv', 'DEEPSEPS FINAL DF.csv', 'los sepsis.csv', 'earlydiagseps.csv', 'diagseps.csv', 'labdss.csv']


In [ ]:
df = pd.read_csv(filepath+ list_of_files[15] )

label_counts = df["diagnosis"].value_counts()
print(label_counts)

diagnosis
SEPSIS                                           25244
PNEUMONIA                                        19213
FEVER                                            10613
FACIAL NUMBNESS                                   7262
AROMEGLEY;BURKITTS LYMPHOMA                       7262
                                                 ...  
ACUTE SUBDURAL HEMATOMA                            105
RECURRENT LEFT CAROTID STENOSIS,PRE HYDRATION      102
SUBDURAL HEMATOMA/S/P FALL                          90
S/P MOTORCYCLE ACCIDENT                             70
PNEUMONIA;TELEMETRY                                 54
Name: count, Length: 95, dtype: int64


In [ ]:
pivoted_data = df.pivot_table(index='subject_id', columns= 'label', values= 'valuenum', aggfunc='first')


pivoted_data.columns = [f"{col[1]}_{col[0]}" for col in pivoted_data.columns]


pivoted_data.reset_index(inplace=True)


pivoted_data = pivoted_data.merge(df[['subject_id','diagnosis']], on='subject_id')


pivoted_data.fillna(0, inplace=True)


print(pivoted_data.sample(1))

        subject_id   _%  5_2  b_A  b_A  b_A  b_A  b_A  b_A  c_A  ...  B_W  \
144398       42281  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

         h_W  r_W  e_Y  o_Y   C_p  H_p    O_p  a_t       diagnosis  
144398  17.0  0.0  0.0  0.0  61.0  7.0  225.0  0.0  UPPER GI BLEED  

[1 rows x 375 columns]


In [ ]:
pivoted_data.shape

(162187, 375)

In [ ]:
pivoted_data['diagnosis'] = pivoted_data['diagnosis'].astype(str)

In [ ]:
pivoted_data.isnull().sum().sort_values(ascending=False)

subject_id    0
x_O           0
h_P           0
h_P           0
e_P           0
             ..
r_C           0
r_C           0
r_C           0
r_C           0
diagnosis     0
Length: 375, dtype: int64

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
pivoted_data['diagnosis'] = le.fit_transform(pivoted_data['diagnosis'])


y = pivoted_data['diagnosis']
X = pivoted_data.iloc[:,:-1]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.30, random_state=20)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred

from sklearn.metrics import f1_score, accuracy_score

acc = accuracy_score(y_test, y_pred)


print(f'KNN Accuracy: {acc}')

KNN Accuracy: 0.4895081899829418


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Bayes F1: {f1}')
print(f'Bayes Accuracy: {acc}')

Bayes F1: 0.46848489414625866
Bayes Accuracy: 0.48981647039480447


In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'DT F1: {f1}')
print(f'DT Accuracy: {acc}')

from sklearn.tree import plot_tree
plt.figure(figsize=(12,10))
plot_tree(model)
plt.show()

DT F1: 0.517241199554373
DT Accuracy: 0.5995437449904433
